  
## Problem Description

There are six end landfills, each with a known demand for a waste material.  Landfill demand can be satisfied from a set of four transfer station depots, or directly from a set of two waste generating centers.  Each transfer depot can support a maximum volume of waste moving through it, and each waste generating center can generate a maximum amount of waste.  There are known costs associated with transporting the waste, from a center to a depot, from a depot to a landfill, or from a center directly to a landfill.

The waste network has two waste generating centers, in NewYork and NewJersey, which generate the waste.  Each has a maximum waste generating volume:

| Center | Waste (tons) |
| --- | --- |
| NewYork | 300,000 |
| NewJersey |  400,000 |

The waste can be shipped from a center to a set of four depots. Each depot has a maximum throughput.

| Depot | Throughput (tons) |
| --- | --- |
| Bronx | 140,000 |
| Brooklyn | 100,000 |
| Queens | 200,000 |
| Staten Island | 80,000 |

Our network has six landfills, each with a given maxiumum demand.

| Landfill | Demand (tons) |
| --- | --- |
| C1 | 100,000 |
| C2 | 20,000 |
| C3 | 80,000 |
| C4 | 70,000 |
| C5 | 120,000 |
| C6 | 40,000 |

Transporation costs are given in the following table (in dollars per ton).  Columns are source cities and rows are destination cities.  

| To | NewYork | NewJersey | Bronx | Brooklyn | Queens | StatenIsland |
| --- | --- | --- | --- | --- | --- | --- |
| Depots |
| Bronx  | 0.7 |   - |
| Brooklyn | 0.7 | 0.5 |
| Queens     | 1.2 | 0.7 |
| Staten Island     | 0.4 | 0.4 |
| Landfills |
| C1 | 1.2 | 2.2 |   - | 1.2 |   - |   - |
| C2 |   - |   - | 1.7 | 0.7 | 1.7 |   - |
| C3 | 1.7 |   - | 0.7 | 0.75 | 2.2 | 0.4 |
| C4 | 2.2 |   - | 1.7 | 1.2|   - | 1.7 |
| C5 |   - |   - |   - | 0.7 | 0.7 | 0.7 |
| C6 | 1.2 |   - | 1.2 |   - | 1.7 | 1.7 |

Question: How to satisfy the demands of the end Landfills while minimizing shipping costs.

---
## Model Formulation

### Sets and Indices

$f \in \text{Centers}=\{\text{NewYork}, \text{NewJersey}\}$

$d \in \text{Depots}=\{\text{Bronx}, \text{Brooklyn}, \text{Queens}, \text{StatenIsland}\}$

$c \in \text{Landfills}=\{\text{C1}, \text{C2}, \text{C3}, \text{C4}, \text{C5}, \text{C6}\}$

$\text{Locations} = \text{Centers} \cup \text{Depots} \cup \text{Landfills}$

### Parameters

$\text{cost}_{s,t} \in \mathbb{R}^+$: Cost of shipping one ton from source $s$ to destination $t$.

$\text{supply}_f \in \mathbb{R}^+$: Maximum possible supply from center $f$ (in tons).

$\text{through}_d \in \mathbb{R}^+$: Maximum possible flow through depot $d$ (in tons).

$\text{demand}_c \in \mathbb{R}^+$: Demand for waste at landfill $c$ (in tons).

### Decision Variables

$\text{flow}_{s,t} \in \mathbb{N}^+$: Quantity of waste (in tons) that is shipped from source $s$ to destionation $t$.


### Objective Function

- **Cost**: Minimize total shipping costs.

\begin{equation}
\text{Minimize} \quad Z = \sum_{(s,t) \in \text{locations} \times \text{locations}}{\text{cost}_{s,t}*\text{flow}_{s,t}}
\end{equation}

### Constraints

- **Center output**: Flow of goods from a factory must respect maximum capacity.

\begin{equation}
\sum_{t \in \text{locations}}{\text{flow}_{f,t}} \leq \text{supply}_{f} \quad \forall f \in \text{Centers}
\end{equation}

- **Landfill demand**: Flow of goods must meet customer demand.

\begin{equation}
\sum_{s \in \text{locations}}{\text{flow}_{s,c}} = \text{demand}_{c} \quad \forall c \in \text{Landfills}
\end{equation}

- **Depot flow**: Flow into a depot equals flow out of the depot.

\begin{equation}
\sum_{s \in \text{locations}}{\text{flow}_{s,d}} = 
\sum_{t \in \text{locations}}{\text{flow}_{d,t}}
\quad \forall d \in \text{Depots}
\end{equation}

- **Depot capacity**: Flow into a depot must respect depot capacity.

\begin{equation}
\sum_{s \in \text{locations}}{\text{flow}_{s,d}} \leq \text{through}_{d}
\quad \forall d \in \text{Depots}
\end{equation}

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

This code snippet imports two Python libraries, `pandas` and `gurobipy`, and assigns them aliases for easier usage in the rest of the program. Additionally, the `# type: ignore` comments are used to suppress type-checking warnings from tools like `mypy`.

1. **`import pandas as pd`**: This imports the `pandas` library, a powerful tool for data manipulation and analysis, and assigns it the alias `pd`. Using an alias like `pd` is a common convention in Python, making the code more concise and readable. For example, instead of writing `pandas.DataFrame`, you can simply write `pd.DataFrame`.

2. **`import gurobipy as gp`**: This imports the `gurobipy` library, which provides a Python interface to the Gurobi Optimizer, and assigns it the alias `gp`. The Gurobi Optimizer is widely used for solving mathematical optimization problems, such as linear programming and mixed-integer programming. The alias `gp` simplifies the usage of the library in the code, allowing you to refer to it more succinctly.

3. **`from gurobipy import GRB`**: This imports the `GRB` module directly from the `gurobipy` library. `GRB` is a collection of constants and parameters used in Gurobi models, such as optimization sense (e.g., minimize or maximize) and variable types (e.g., continuous, integer, or binary). Importing it directly allows you to use these constants without needing to prefix them with `gp.`.

4. **`# type: ignore`**: These comments instruct static type-checking tools like `mypy` to ignore potential type-related issues in these import statements. This is often used when the imported libraries are not fully compatible with the type-checking tool or when type hints are not available for the library. It ensures that type-checking does not raise unnecessary warnings or errors, allowing the code to pass validation smoothly.

In summary, this snippet sets up the necessary imports for working with data (`pandas`) and optimization models (`gurobipy`) while ensuring that type-checking tools do not interfere with the imports. The use of aliases (`pd` and `gp`) and direct imports (`GRB`) improves code readability and convenience.

In [ ]:
import pandas          as     pd  # type: ignore
import gurobipy        as     gp  # type: ignore
from   gurobipy        import GRB # type: ignore
from   IPython.display import display
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Input Data
We define all the input data for the model.

This code defines the input data for a supply chain optimization problem, capturing the constraints and costs associated with transporting waste from generating centers to landfills via depots. It uses Python dictionaries to store the supply, throughput, and demand limits, as well as the transportation costs between locations.

The `supply` dictionary specifies the maximum waste generation capacity (in tons) for two centers: New York (300,000 tons) and New Jersey (400,000 tons). These values represent the upper limits of waste that can be supplied by each center.

The `through` dictionary defines the maximum throughput capacity (in tons) for four depots: Bronx (140,000 tons), Brooklyn (100,000 tons), Queens (200,000 tons), and Staten Island (80,000 tons). These values indicate the maximum volume of waste that can pass through each depot.

The `demand` dictionary outlines the waste demand (in tons) for six landfills: C1 (100,000 tons), C2 (20,000 tons), C3 (80,000 tons), C4 (70,000 tons), C5 (120,000 tons), and C6 (40,000 tons). These values represent the amount of waste each landfill requires to meet its demand.

The `gp.multidict` function is used to define the transportation costs between various locations. It creates two outputs: `arcs`, which is a list of all valid source-destination pairs (e.g., ('NewYork', 'Bronx')), and `cost`, which is a dictionary mapping each arc to its associated cost (in dollars per ton). For example, the cost of transporting waste from New York to Bronx is $0.7 per ton, while the cost from New York to Staten Island is $0.4 per ton. The costs also include direct shipments from centers to landfills and from depots to landfills.

This structured data will be used in the optimization model to determine the optimal flow of waste that minimizes transportation costs while satisfying all supply, throughput, and demand constraints.

In [2]:
# Create dictionaries to capture center supply limits, depot throughput limits, and landfill demand.
supply  = dict({'NewYork'     : 300000,
                'NewJersey'   : 400000})
through = dict({'Bronx'       : 140000,
                'Brooklyn'    : 100000,
                'Queens'      : 200000,
                'StatenIsland':  80000})
demand  = dict({'C1'          : 100000,
                'C2'          :  20000,
                'C3'          :  80000,
                'C4'          :  70000,
                'C5'          : 120000,
                'C6'          :  40000})
# Create a dictionary to capture shipping costs.
arcs, cost = gp.multidict({
    ('NewYork',      'Bronx'):        0.7,
    ('NewYork',      'Brooklyn'):     0.7,
    ('NewYork',      'Queens'):       1.2,
    ('NewYork',      'StatenIsland'): 0.4,
    ('NewYork',      'C1'):           1.2,
    ('NewYork',      'C3'):           1.7,
    ('NewYork',      'C4'):           2.2,
    ('NewYork',      'C6'):           1.2,
    ('NewJersey',    'Brooklyn'):     0.5,
    ('NewJersey',    'Queens'):       0.7,
    ('NewJersey',    'StatenIsland'): 0.4,
    ('NewJersey',    'C1'):           2.2,
    ('Bronx',        'C2'):           1.7,
    ('Bronx',        'C3'):           0.7,
    ('Bronx',        'C4'):           1.7,
    ('Bronx',        'C6'):           1.2,
    ('Brooklyn',     'C1'):           1.2,
    ('Brooklyn',     'C2'):           0.7,
    ('Brooklyn',     'C3'):           0.7,
    ('Brooklyn',     'C4'):           1.2,
    ('Brooklyn',     'C5'):           0.7,
    ('Queens',       'C2'):           1.7,
    ('Queens',       'C3'):           2.2,
    ('Queens',       'C5'):           0.7,
    ('Queens',       'C6'):           1.7,
    ('StatenIsland', 'C3'):           0.4,
    ('StatenIsland', 'C4'):           1.7,
    ('StatenIsland', 'C5'):           0.7,
    ('StatenIsland', 'C6'):           1.7
})

## Model Deployment

Create a model and  variables. The variables simply capture the amount of waste that flows along each allowed path between a source and destination.  Objective coefficients are provided here (in $\text{cost}$) , so we don't need to provide an optimization objective later.

This code snippet initializes a Gurobi optimization model and defines decision variables for the supply chain optimization problem.

1. **`model = gp.Model('SupplyNetworkDesign')`**: This creates a new Gurobi model instance named `'SupplyNetworkDesign'`. The model serves as a container for all the components of the optimization problem, including variables, constraints, and the objective function. Naming the model helps in identifying it, especially when working with multiple models.

2. **`flow = model.addVars(arcs, obj=cost, name="flow")`**: This defines a set of decision variables, one for each valid source-destination pair (arc) in the supply chain. The `addVars` method creates these variables with the following key parameters:
   - **`arcs`**: This is the set of indices for the variables, representing all valid source-destination pairs (e.g., `('NewYork', 'Bronx')`).
   - **`obj=cost`**: This assigns the transportation cost (in dollars per ton) as the objective coefficient for each variable. These coefficients will be used in the objective function to minimize the total shipping cost.
   - **`name="flow"`**: This assigns the name `'flow'` to the variables, making them easier to reference in the model and in the solution output.

The resulting `flow` object is a `tupledict`, which is a Gurobi-specific data structure that maps each arc (e.g., `('NewYork', 'Bronx')`) to its corresponding decision variable. Each variable represents the quantity of waste (in tons) transported along a specific arc.

In summary, this code sets up the optimization model and defines the key decision variables that will be optimized to minimize the total transportation cost while satisfying the problem's constraints.

In [3]:
model = gp.Model('SupplyNetworkDesign')
flow  = model.addVars(arcs, obj=cost, name="flow")

Restricted license - for non-production use only - expires 2026-11-23


First constraints require the total flow along arcs leaving a center to be at most as large as the supply capacity of that center.

This code defines a set of constraints for the optimization model to ensure that the total flow of waste leaving each waste-generating center does not exceed its supply capacity.

1. **`centers = supply.keys()`**: The `supply` dictionary contains the maximum waste generation capacity for each center (e.g., New York and New Jersey). The `keys()` method retrieves the names of these centers as a view object, which can be iterated over. This allows the code to dynamically handle all centers defined in the `supply` dictionary.

2. **`gp.quicksum(flow.select(center, '*'))`**: The `flow` object is a `tupledict` containing decision variables that represent the amount of waste transported along specific arcs. The `select(center, '*')` method filters the `flow` variables to include only those arcs where the source is the specified `center` (e.g., New York). The `gp.quicksum()` function then computes the sum of these selected variables, representing the total flow of waste leaving the center.

3. **`<= supply[center]`**: This ensures that the total flow of waste leaving a center does not exceed its supply capacity, as defined in the `supply` dictionary. For example, if New York has a supply capacity of 300,000 tons, the total flow leaving New York must be less than or equal to 300,000.

4. **`model.addConstrs(...)`**: The `addConstrs` method adds a set of constraints to the optimization model. The constraints are generated using a Python generator expression, which iterates over all centers and creates one constraint for each. The `name="center"` parameter assigns a common name to this group of constraints, making them easier to reference or debug later.

In summary, this code ensures that the waste flow leaving each center respects its supply capacity. It dynamically generates constraints for all centers in the `supply` dictionary, contributing to the feasibility of the optimization model by enforcing supply limits.

In [4]:
# Center capacity limits
centers     = supply.keys()
center_flow = model.addConstrs((gp.quicksum(flow.select(center, '*')) <= supply[center]
                                 for center in centers), name="center")

Next constraints require the total flow along arcs entering a landfill to be equal to the demand from that landfill.

This code defines a set of constraints for the optimization model to ensure that the total flow of waste entering each landfill matches its demand.

1. **`landfills = demand.keys()`**: The `demand` dictionary specifies the waste demand (in tons) for each landfill (e.g., C1, C2, etc.). The `keys()` method retrieves the names of these landfills as a view object, which can be iterated over. This allows the code to dynamically handle all landfills defined in the `demand` dictionary.

2. **`gp.quicksum(flow.select('*', landfill))`**: The `flow` object is a `tupledict` containing decision variables that represent the amount of waste transported along specific arcs. The `select('*', landfill)` method filters the `flow` variables to include only those arcs where the destination is the specified `landfill` (e.g., C1). The `gp.quicksum()` function then computes the sum of these selected variables, representing the total flow of waste entering the landfill.

3. **`== demand[landfill]`**: This ensures that the total flow of waste entering a landfill is exactly equal to its demand, as defined in the `demand` dictionary. For example, if landfill C1 has a demand of 100,000 tons, the total flow entering C1 must equal 100,000.

4. **`model.addConstrs(...)`**: The `addConstrs` method adds a set of constraints to the optimization model. The constraints are generated using a Python generator expression, which iterates over all landfills and creates one constraint for each. The `name="landfill"` parameter assigns a common name to this group of constraints, making them easier to reference or debug later.

In summary, this code ensures that the waste flow entering each landfill satisfies its demand. It dynamically generates constraints for all landfills in the `demand` dictionary, contributing to the feasibility of the optimization model by enforcing demand satisfaction.

In [5]:
# landfill demand
landfills     = demand.keys()
landfill_flow = model.addConstrs((gp.quicksum(flow.select('*', landfill)) == demand[landfill]
                                  for landfill in landfills), name="landfill")

Final constraints relate to depots.  The first constraints require that the total amount of waste entering the depot must equal the total amount leaving.

This code defines a set of constraints for the optimization model to ensure **flow conservation** at each depot. Flow conservation means that the total amount of waste entering a depot must equal the total amount of waste leaving the depot.

1. **`depots = through.keys()`**: The `through` dictionary contains the maximum throughput capacity for each depot (e.g., Bronx, Brooklyn, etc.). The `keys()` method retrieves the names of these depots as a view object, which can be iterated over. This allows the code to dynamically handle all depots defined in the `through` dictionary.

2. **`gp.quicksum(flow.select(depot, '*'))`**: The `flow` object is a `tupledict` containing decision variables that represent the amount of waste transported along specific arcs. The `select(depot, '*')` method filters the `flow` variables to include only those arcs where the source is the specified `depot` (e.g., Bronx). The `gp.quicksum()` function then computes the sum of these selected variables, representing the total flow of waste leaving the depot.

3. **`gp.quicksum(flow.select('*', depot))`**: Similarly, this computes the total flow of waste entering the depot by filtering the `flow` variables to include only those arcs where the destination is the specified `depot`.

4. **`==`**: This ensures that the total flow entering a depot is equal to the total flow leaving the depot. This is a key constraint for maintaining flow balance at each depot.

5. **`model.addConstrs(...)`**: The `addConstrs` method adds a set of constraints to the optimization model. The constraints are generated using a Python generator expression, which iterates over all depots and creates one constraint for each. The `name="depot"` parameter assigns a common name to this group of constraints, making them easier to reference or debug later.

In summary, this code enforces flow conservation at each depot by ensuring that the total waste entering a depot equals the total waste leaving it. This is a critical constraint for maintaining the integrity of the supply chain network and ensuring that waste is properly routed through the depots.

In [6]:
# Depot flow conservation
depots     = through.keys()
depot_flow = model.addConstrs((gp.quicksum(flow.select(depot, '*')) == gp.quicksum(flow.select('*', depot))
                               for depot in depots), name="depot")

The second set limits the waste passing through the depot to be at most equal the throughput of that depot.

This code defines a set of constraints for the optimization model to ensure that the total amount of waste entering each depot does not exceed its throughput capacity.

1. **`gp.quicksum(flow.select('*', depot))`**: The `flow` object is a `tupledict` containing decision variables that represent the amount of waste transported along specific arcs. The `select('*', depot)` method filters the `flow` variables to include only those arcs where the destination is the specified `depot` (e.g., Bronx). This effectively identifies all incoming flows to the depot. The `gp.quicksum()` function then computes the sum of these selected variables, representing the total flow of waste entering the depot.

2. **`<= through[depot]`**: The `through` dictionary specifies the maximum throughput capacity for each depot. This constraint ensures that the total flow of waste entering a depot does not exceed its capacity. For example, if the Queens depot has a throughput capacity of 200,000 tons, the total flow entering Queens must be less than or equal to 200,000.

3. **`model.addConstrs(...)`**: The `addConstrs` method adds a set of constraints to the optimization model. It takes a generator expression that iterates over all depots and creates one constraint for each depot. Each constraint ensures that the total incoming flow to a depot is within its capacity. The `name="depot_capacity"` parameter assigns a common name to this group of constraints, making them easier to reference or debug later.

In summary, this code enforces throughput capacity limits at each depot by ensuring that the total waste entering a depot does not exceed its maximum allowable capacity. This is a critical constraint for maintaining the feasibility of the optimization model and ensuring that depots are not overloaded.

In [7]:
# Depot throughput
depot_capacity = model.addConstrs((gp.quicksum(flow.select('*', depot)) <= through[depot]
                                   for depot in depots), name="depot_capacity")

Optimize the model

The `model.optimize()` method is a key function in the Gurobi optimization process. It triggers the solver to find the optimal solution for the mathematical model defined earlier. This method uses the constraints, decision variables, and objective function specified in the model to compute the best solution.

1. **Optimization Process**: When `model.optimize()` is called, Gurobi begins solving the optimization problem using its internal algorithms. Depending on the problem type (e.g., linear programming, mixed-integer programming), it applies techniques such as the simplex method, branch-and-bound, or cutting planes to find the optimal solution.

2. **Objective**: The goal of the optimization is to minimize or maximize the objective function defined in the model. For example, in this case, the objective is likely to minimize the total transportation cost, as specified in the earlier parts of the code.

3. **Constraints Enforcement**: During the optimization, Gurobi ensures that all constraints defined in the model are satisfied. These constraints include supply limits, demand satisfaction, depot throughput, and flow conservation, ensuring the solution is feasible within the problem's parameters.

4. **Callback Option**: The `optimize()` method optionally accepts a `callback` function, which can be used to monitor or modify the optimization process dynamically. For example, a callback can be used to log intermediate results or implement custom stopping criteria.

In summary, `model.optimize()` is the command that initiates the optimization process, allowing Gurobi to compute the optimal solution for the defined problem while respecting all constraints and minimizing the objective function. Once the optimization is complete, the results can be analyzed to extract insights or make decisions.

In [24]:
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.5.0 24F5053j)

CPU model: Apple M4 Max
Thread count: 16 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 16 rows, 29 columns and 65 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.410000000e+05


---
## Analysis

Waste demand from all of our landfillss can be satisfied for a total cost of $\$541,000$. The optimal plan is as follows.

This code processes the results of an optimization model to generate two structured DataFrames: `product_flow` and `product_cost`. These DataFrames summarize the flow of waste along arcs and the associated costs, respectively. The results are then displayed for analysis.

### **Flow DataFrame (`product_flow`)**
1. **Initialization**:  
   An empty DataFrame `product_flow` is created with columns `"From"`, `"To"`, and `"Flow"`. This will store the source, destination, and flow value for each arc.

2. **Iterating Through Arcs**:  
   The `for` loop iterates over all arcs in the `arcs` list. Each arc represents a valid source-destination pair in the network.

3. **Filtering Significant Flows**:  
   The condition `if flow[arc].x > 1e-6` ensures that only arcs with a flow value greater than a small threshold are included. This avoids adding rows for arcs with negligible or zero flow.

4. **Adding Rows**:  
   For each arc with significant flow, a new DataFrame `new_row` is created containing the source (`arc[0]`), destination (`arc[1]`), and flow value (`flow[arc].x`). This row is appended to `product_flow` using `pd.concat`, with `ignore_index=True` to reset the index.

5. **Custom Index**:  
   After the loop, the index of `product_flow` is replaced with empty strings (`''`) for aesthetic purposes when displaying the DataFrame.

### **Cost DataFrame (`product_cost`)**
1. **Initialization**:  
   An empty DataFrame `product_cost` is created with columns `"From"`, `"To"`, `"cost"`, `"Flow"`, and `"total"`. This will store the source, destination, cost per unit, flow value, and total cost for each arc.

2. **Iterating Through Arcs**:  
   Similar to `product_flow`, the `for` loop iterates over all arcs, filtering those with significant flow (`flow[arc].x > 1e-6`).

3. **Calculating Total Cost**:  
   For each arc, the total cost is calculated as `cost[arc] * flow[arc].x` and added to the cumulative variable `k`.

4. **Adding Rows**:  
   A new DataFrame `new_row` is created for each arc, containing the source, destination, cost per unit, flow value, and total cost. This row is appended to `product_cost` using `pd.concat`.

5. **Custom Index**:  
   The index of `product_cost` is also replaced with empty strings for display purposes.

### **Displaying Results**
1. **Flow DataFrame**:  
   The `product_flow` DataFrame is displayed using `display(product_flow)` after printing a header.

2. **Cost DataFrame**:  
   The `product_cost` DataFrame is displayed similarly, showing the cost breakdown for each arc.

3. **Total Cost**:  
   The cumulative total cost `k` is printed, summarizing the overall cost of the flows.

### **Summary**
This code dynamically constructs two DataFrames to summarize the optimization results. `product_flow` focuses on the flow of waste, while `product_cost` provides a detailed cost analysis. The use of filtering, concatenation, and custom indexing ensures that only meaningful data is included, making the results easy to interpret and analyze.

In [22]:
product_flow = pd.DataFrame(columns=["From", "To", "Flow"])
for arc in arcs:
    if flow[arc].x > 1e-6:
        # Create a new DataFrame for the current row
        new_row = pd.DataFrame({"From": [arc[0]], "To": [arc[1]], "Flow": [flow[arc].x]})
        # Concatenate the new row with the existing DataFrame
        product_flow = pd.concat([product_flow, new_row], ignore_index=True)  
product_flow.index=[''] * len(product_flow)
product_flow

k = 0 
product_cost = pd.DataFrame(columns=["From", "To", "cost", "Flow", "total"])
for arc in arcs:
    if flow[arc].x > 1e-6:
        k = k + cost[arc] * flow[arc].X
        # Create a new DataFrame for the current row
        new_row = pd.DataFrame({"From": [arc[0]], "To": [arc[1]], "cost": [cost[arc]], "Flow": [flow[arc].x], "total": [cost[arc] * flow[arc].x]})
        # Concatenate the new row with the existing DataFrame
        product_cost = pd.concat([product_cost, new_row], ignore_index=True)  
product_cost.index = [''] * len(product_cost)

print("Flow")
print("------------------------------------")
display(product_flow)

print("Cost")
print("---------------------------------------------------")
display(product_cost)

print("Total flow: ", k)

Flow
------------------------------------


,From,To,Flow
,NewYork,C1,100000.0
,NewYork,C6,40000.0
,NewJersey,Brooklyn,100000.0
,NewJersey,Queens,110000.0
,NewJersey,StatenIsland,80000.0
,Brooklyn,C2,20000.0
,Brooklyn,C4,70000.0
,Brooklyn,C5,10000.0
,Queens,C5,110000.0
,StatenIsland,C3,80000.0


Cost
---------------------------------------------------


,From,To,cost,Flow,total
,NewYork,C1,1.2,100000.0,120000.0
,NewYork,C6,1.2,40000.0,48000.0
,NewJersey,Brooklyn,0.5,100000.0,50000.0
,NewJersey,Queens,0.7,110000.0,77000.0
,NewJersey,StatenIsland,0.4,80000.0,32000.0
,Brooklyn,C2,0.7,20000.0,14000.0
,Brooklyn,C4,1.2,70000.0,84000.0
,Brooklyn,C5,0.7,10000.0,7000.0
,Queens,C5,0.7,110000.0,77000.0
,StatenIsland,C3,0.4,80000.0,32000.0


Total flow:  541000.0
